# Differential Privacy Variants in Deep Learning

- Deep learning models require large volumes of data to produce reasonable results after training. This data may contain sensitive information which can be exposed through the models.  

- In this project, we have implemented differential privacy in a deep learning model.  Specifically, we perform experiments with the DP-SGD algorithm using the Gaussian Mechanism, Renyi differential privacy and  zero-concentrated differential privacy using a Convolutional Neural Network (CNN). 

- We computed our differentially private gradient descent algorithm using the following steps: 

    1. Calculate gradients for each example (per_sample_gradients)
    2. Clip the gradients to have bounded L2 norm 
    3. Average the clipped gradients
    4. Then add noise to the average of the gradients

- The main difference between the variants of differential privacy used here is in the scale/amount of noise added. All variants draw noise from the Gaussian probability distribution function. 

- All experiments have been implemented on the MNIST dataset and their performances evaluated.  

- Our results show that with an epsilon/epsilon_bar of 10, Renyi DP achieves the best performance with an accuracy of 68%, followed by zero-concerntrated differential privacy with 67% and finally the gaussian mechanism.

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import collections
import copy

# from opacus import PrivacyEngine
from torch.nn.utils import clip_grad_norm_

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import warnings
warnings.simplefilter("ignore")
device

device(type='cuda')

In [3]:
# Defining Hyperparameters
MAX_GRAD_NORM = 0.1 # The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step.
EPSILON = 10.0
DELTA = 1e-5 # The target δ of the (ϵ,δ)-differential privacy guarantee.
EPOCHS = 10 
NOISE_MULTIPLIER = 1.3
ALPHA = 2
RHO = 0.1
LR = 1e-3
BATCH_SIZE = 600

In [4]:
train_dataset = datasets.MNIST('../mnist',
                   train=True,
                   download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,)),]),)

train_loader = torch.utils.data.DataLoader(
                  train_dataset,
                  batch_size=BATCH_SIZE,
                  )


test_dataset =  datasets.MNIST('../mnist', 
                       train=False, 
                       transform=transforms.Compose([transforms.ToTensor(), 
                                                     transforms.Normalize((0.1307,), (0.3081,)),]),)
test_loader = torch.utils.data.DataLoader( 
                    test_dataset, 
                    batch_size=BATCH_SIZE,
                    shuffle=False,
                  )

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../mnist/MNIST/raw



In [5]:
# defining the deep learning classifier
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.pool = nn.MaxPool2d(2,1)
        self.conv2 =  nn.Conv2d(16, 32, 4, 2) 
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        #self.fc1_drop = nn.Dropout(p=0.4)
        self.fc2 = nn.Linear(32,10)
        

    ## TODO: define the feedforward behavior
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        #x = self.fc1_drop(x)
        x = self.fc2(x)                
        return x

In [6]:
# compute per example gradient by feeding an example each pass and recording its gradient
def per_example_gradient(data, target, model, criterion, g_dict, optimizer):
    loss_val = 0
    batch_size = data.size()[0]

    for i in range(batch_size):
        data_i, target_i = data[i].unsqueeze(0), target[i].unsqueeze(0)
        output = model(data_i)
        loss = criterion(output, target_i)
        loss_val += loss.item()
        optimizer.zero_grad()
        loss.backward()

        for name, param in model.named_parameters():
            g_dict[name].append(copy.deepcopy(param.grad.data))

    return g_dict, loss_val/batch_size

In [7]:
def santinizer(g_dict, variant, epsilon_iter, device):
    # this is the scale of the Gaussian noise to be added to the batch gradient
    if(variant == "rdp"): # Renyi
      sigma = np.sqrt((MAX_GRAD_NORM**2 * ALPHA) / (2 * epsilon_iter))
    elif (variant == "zdp"): #Zero Concernttrated Gradient
      rho_i = epsilon_iter
      sigma = np.sqrt((MAX_GRAD_NORM**2) / (2 * rho_i))
    elif(variant == "dp"): # Regular DP
      sigma = MAX_GRAD_NORM * np.sqrt(2*np.log(1.25/DELTA)) / epsilon_iter

    batch_size = BATCH_SIZE
    noise_dist = torch.distributions.Normal(loc=0.0, scale=sigma*MAX_GRAD_NORM/batch_size)
    #noise_dist = torch.distributions.Normal(loc=0.0, scale = sigma)

    # compute and clip norm for every sample
    for var in g_dict:
        gradients = g_dict[var]

        assert len(gradients) == batch_size
        for i in range(batch_size):
            l2_norm = torch.norm(gradients[i]) #norm
            inv_norm = max(1.0, l2_norm/MAX_GRAD_NORM)  # clip
            gradients[i] = gradients[i] / inv_norm

        # Average clipped gradients
        g_cat = torch.stack(gradients, dim=0)
        g_mean = torch.mean(g_cat, dim=0).to(device) # average grads

        # Add noise to average of gradients
        noise = noise_dist.sample(g_mean.size()).to(device) 
        g_dict[var] = g_mean + noise  #add noise
    return g_dict

def resign_gradient(g_dict, model):
    '''
    assign gradient from g_dict to model parameter
    '''
    for name, param in model.named_parameters():
        param.grad = g_dict[name]
    return model

def adjust_learning_rate(optimizer, epoch, init_lr=0.1, saturate_epoch=10, stop_lr=0.052):
    '''
    linearly adjust learning_rate, accoring to tensorflow's implementation
    '''
    step = (init_lr - stop_lr) / (saturate_epoch-1)
    if epoch < saturate_epoch:
        lr = init_lr - step * epoch
    else:
        lr = stop_lr
    for param_group in optimizer.param_groups:
            param_group['lr'] = lr

In [8]:
#  Train model 
def dp_train(model, train_loader, optimizer, criterion, epsilon, device, variant):
    iters = EPOCHS * BATCH_SIZE
    epsilon_iter = epsilon / iters
    rho_i =  RHO / iters

    # train phase
    for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
        adjust_learning_rate(optimizer, epoch)

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target =  Variable(data).to(device), Variable(target).to(device)
            # computer per sample gradients
            g_dict = collections.defaultdict(list)
            g_dict, loss_val = per_example_gradient(data, target, model, criterion, g_dict, optimizer)
            
            # clip and add noise to l2 norm of grads
            g_dict = santinizer(g_dict, variant, epsilon_iter, device)

            optimizer.zero_grad()

            # reset grads
            model = resign_gradient(g_dict, model)
            optimizer.step()

            if batch_idx % 10000 == 0:
                output = model(data)
                pred = output.data.max(1)[1]
                correct = pred.eq(target.data).sum()

                print('\tEpoch {} => Loss: {:.3f} \tAcc: {:.3f} '.format(epoch, loss_val, correct.item()/BATCH_SIZE))

    # test phase
    total_acc = dp_test(model, test_loader, criterion)
    return model

In [9]:
# Evaluate model's performance on test dataset
def dp_test(model, dataloader, criterion):
    test_loss = 0
    correct = 0
    for data, target in dataloader:
        data, target = Variable(data).to(device), Variable(target).to(device)
        
        output = model(data)
        test_loss += criterion(output, target).mean()
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(dataloader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataloader.dataset), 100.0 * correct / len(dataloader.dataset)
        )
    )

    return 100.0 * correct / len(dataloader.dataset)


## Training Without Differential Privacy

In [33]:
# training cnn model
def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    accs = []
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # compute output
        output = model(data)
        loss = criterion(output, target)

        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()
        
        # measure accuracy and record loss
        acc = accuracy(preds, labels)
        losses.append(loss.item())
        accs.append(acc)

        loss.backward()
        optimizer.step()
    
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"Accuracy: {np.mean(accs):.6f} "
    )

def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []
    correct = 0
    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).sum()
            losses.append(loss.item())
            
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            np.mean(losses), correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

    return 100.0 * correct / len(test_loader.dataset)

def accuracy(preds, labels):
    return (preds == labels).mean()

In [19]:
%%time
basic_model = Classifier()
basic_model.to(device)
optimizer = torch.optim.SGD(basic_model.parameters(), lr=0.01)
for epoch in range(10):
    train(basic_model, train_loader, optimizer, epoch, device="cuda") 

100%|██████████| 100/100 [00:16<00:00,  6.13it/s]


Train Epoch: 0 	Loss: 2.238898 Accuracy: 0.306383 


100%|██████████| 100/100 [00:16<00:00,  6.18it/s]


Train Epoch: 1 	Loss: 1.708992 Accuracy: 0.650467 


100%|██████████| 100/100 [00:16<00:00,  6.09it/s]


...

Train Epoch: 7 	Loss: 0.206112 Accuracy: 0.938217 


100%|██████████| 100/100 [00:16<00:00,  6.02it/s]


Train Epoch: 8 	Loss: 0.188413 Accuracy: 0.943400 


100%|██████████| 100/100 [00:16<00:00,  5.92it/s]

Train Epoch: 9 	Loss: 0.174269 Accuracy: 0.947600 
CPU times: user 2min 40s, sys: 2.79 s, total: 2min 42s
Wall time: 2min 43s


In [34]:
acc = test(basic_model, test_loader, device)


Test set: Average loss: 0.1559, Accuracy: 9519/10000 (95%)



## Differential Privacy (Gaussian Mechanism)

In [ ]:
%%time
dp_model = Classifier().to(device)
criterion = nn.CrossEntropyLoss(reduce=False)
optimizer = torch.optim.SGD(dp_model.parameters(), lr=LR)
epsilon = 10

dp_model = dp_train(dp_model, train_loader, optimizer, criterion, epsilon, device, "dp")

Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

	Epoch 0 => Loss: 2.319 	Acc: 0.073 


Epoch:  10%|█         | 1/10 [04:01<36:10, 241.19s/epoch]

	Epoch 1 => Loss: 2.321 	Acc: 0.160 


Epoch:  20%|██        | 2/10 [08:00<31:58, 239.85s/epoch]

...

	Epoch 6 => Loss: 2.259 	Acc: 0.270 


Epoch:  70%|███████   | 7/10 [27:56<11:57, 239.19s/epoch]

	Epoch 7 => Loss: 2.253 	Acc: 0.265 


Epoch:  80%|████████  | 8/10 [31:54<07:57, 238.64s/epoch]

	Epoch 8 => Loss: 2.219 	Acc: 0.273 


In [ ]:
total_acc = dp_test(dp_model, test_loader, criterion)


Test set: Average loss: 0.0035, Accuracy: 3069/10000 (31%)



## Renyi Differential Privacy

In [ ]:
%%time
rdp_model = Classifier().to(device)
criterion = nn.CrossEntropyLoss(reduce=False)
optimizer = torch.optim.SGD(rdp_model.parameters(), lr=LR)
epsilon = 10

rdp_model = dp_train(rdp_model, train_loader, optimizer, criterion, epsilon, device, "rdp")

Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

	Epoch 0 => Loss: 2.326 	Acc: 0.093 


Epoch:  10%|█         | 1/10 [04:02<36:21, 242.41s/epoch]

	Epoch 1 => Loss: 2.277 	Acc: 0.220 


Epoch:  20%|██        | 2/10 [08:03<32:11, 241.39s/epoch]

...

	Epoch 4 => Loss: 2.010 	Acc: 0.542 


Epoch:  50%|█████     | 5/10 [20:11<20:13, 242.64s/epoch]

	Epoch 5 => Loss: 1.838 	Acc: 0.630 


Epoch:  60%|██████    | 6/10 [24:13<16:10, 242.62s/epoch]

	Epoch 6 => Loss: 1.637 	Acc: 0.677 


In [ ]:
rdp_total_acc = dp_test(rdp_model, test_loader, criterion)


Test set: Average loss: 0.0018, Accuracy: 6757/10000 (68%)



## Zero Concerntrated Differential Privacy

In [ ]:
%%time
zdp_model = Classifier().to(device)
criterion = nn.CrossEntropyLoss(reduce=False)
optimizer = torch.optim.SGD(zdp_model.parameters(), lr=LR)
epsilon = 10

zdp_model = dp_train(zdp_model, train_loader, optimizer, criterion, epsilon, device, "zdp")


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

	Epoch 0 => Loss: 2.313 	Acc: 0.107 


Epoch:  10%|█         | 1/10 [04:01<36:10, 241.21s/epoch]

	Epoch 1 => Loss: 2.272 	Acc: 0.145 


Epoch:  20%|██        | 2/10 [08:03<32:15, 241.97s/epoch]

...

	Epoch 8 => Loss: 1.494 	Acc: 0.667 


Epoch:  90%|█████████ | 9/10 [36:08<03:59, 239.74s/epoch]

	Epoch 9 => Loss: 1.384 	Acc: 0.670 


Epoch: 100%|██████████| 10/10 [40:06<00:00, 240.68s/epoch]



Test set: Average loss: 0.0022, Accuracy: 6661/10000 (67%)

CPU times: user 39min 26s, sys: 56.5 s, total: 40min 23s
Wall time: 40min 8s


In [ ]:
zdp_total_acc = dp_test(zdp_model, test_loader, criterion)


Test set: Average loss: 0.0022, Accuracy: 6661/10000 (67%)



### DP For a Very High Value of Epsilon

In [ ]:
%%time
dp_model = Classifier().to(device)
criterion = nn.CrossEntropyLoss(reduce=False)
optimizer = torch.optim.SGD(dp_model.parameters(), lr=LR)
epsilon = 600 # equal to batch size

dp_model= dp_train(dp_model, train_loader, optimizer, criterion, epsilon, device, "dp")


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]



Loss: 2.323 	Acc: 0.103 


Epoch:  10%|█         | 1/10 [04:29<40:28, 269.86s/epoch]



Loss: 0.373 	Acc: 0.908 


Epoch:  20%|██        | 2/10 [08:57<35:49, 268.67s/epoch]

...



Loss: 0.156 	Acc: 0.965 


Epoch:  90%|█████████ | 9/10 [40:38<04:30, 270.95s/epoch]



Loss: 0.149 	Acc: 0.967 


Epoch: 100%|██████████| 10/10 [45:04<00:00, 270.47s/epoch]



Test set: Average loss: 0.0002, Accuracy: 9693/10000 (97%)

CPU times: user 44min 15s, sys: 1min 7s, total: 45min 23s
Wall time: 45min 9s


## Multiple Epsilons and Epsilon_bar

Could not finish running code on time. We used Google Colab for our experiments and we were logged out before the entire code could run. 

The code ran for several values of epsilon in DP-SGD with the Gaussian mechanism. We can see that the accuracy increases as epsilon/epsilon_bar increases. However, the privacy decreases.

In [10]:
# def build_model(model, epochs=10, batch_size=600, learning_rate=0.1, C=4.0, sigma=2.0):
def trains( train_loader, optimizer, criterion, epsilon, device, variant):
    
    model = Classifier().to(device)
    criterion = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.SGD(model.parameters(), lr=LR)


    iters = EPOCHS * BATCH_SIZE
    epsilon_iter = epsilon / iters
    rho_i =  RHO / iters

    # train phase
    #for epoch in range(EPOCHS):
    for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
        adjust_learning_rate(optimizer, epoch)

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target =  Variable(data).to(device), Variable(target).to(device)
            # computer per sample gradients
            g_dict = collections.defaultdict(list)
            g_dict, loss_val = per_example_gradient(data, target, model, criterion, g_dict, optimizer)
            
            # clip and add noise to l2 norm of grads
            #g_dict = santinizer(g_dict, C, sigma, BATCH_SIZE, device)
            g_dict = santinizer(g_dict, variant, epsilon_iter, device)

            optimizer.zero_grad()

            # reset grads
            model = resign_gradient(g_dict, model)
            optimizer.step()

            if batch_idx % 10000 == 0:
                output = model(data)
                pred = output.data.max(1)[1]
                correct = pred.eq(target.data).sum()

                print('\tEpoch {} => Loss: {:.3f} \tAcc: {:.3f} '.format(epoch, loss_val, correct.item()/BATCH_SIZE))
        

    # test phase
    total_acc = dp_test(model, test_loader, criterion)
    return model

In [11]:
# Save results to google drive incase we are logged out.
import numpy as np
from numpy import savetxt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# # Multiple Epsilons
epsilons = [0.1, 1.0, 10.0, 50.0, 100.0, 200.0]

In [ ]:
%%time
dp_model = Classifier().to(device)
criterion = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.SGD(dp_model.parameters(), lr=LR)

# dp
dp_experiment_results = [trains( train_loader, optimizer, criterion, e, device, "dp") for e in epsilons]
dp_acc = [dp_test(model, test_loader, device) for model in dp_experiment_results]
np.savetxt('/content/drive/My Drive/dp_acc2.csv', dp_acc, fmt='%s')

Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

	Epoch 0 => Loss: 2.322 	Acc: 0.103 


Epoch:  10%|█         | 1/10 [04:32<40:55, 272.78s/epoch]

	Epoch 1 => Loss: 11861478.931 	Acc: 0.058 


Epoch:  20%|██        | 2/10 [09:03<36:11, 271.40s/epoch]

...

	Epoch 6 => Loss: 1.637 	Acc: 0.477 


Epoch:  70%|███████   | 7/10 [30:48<13:11, 263.94s/epoch]

	Epoch 7 => Loss: 1.482 	Acc: 0.548 


Epoch:  80%|████████  | 8/10 [35:15<08:49, 264.98s/epoch]

	Epoch 8 => Loss: 1.368 	Acc: 0.560 


In [36]:
%%time
rdp_model = Classifier().to(device)
criterion = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.SGD(rdp_model.parameters(), lr=LR)

rdp_experiment_results = [dp_train(rdp_model, train_loader, optimizer, criterion, e, device, "rdp") for e in epsilons]
rdp_acc = [dp_test(model, test_loader, device) for model in rdp_experiment_results]
np.savetxt('/content/drive/My Drive/rdp_acc2.csv', rdp_acc, fmt='%s')

In [ ]:
%%time
zdp_model = Classifier().to(device)
criterion = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.SGD(zdp_model.parameters(), lr=LR)

# zdp
zdp_experiment_results = [dp_train(zdp_model, train_loader, optimizer, criterion, e, device, "zdp") for e in epsilons]
zdp_acc = [dp_test(model, test_loader, device) for model in zdp_experiment_results]
np.savetxt('/content/drive/My Drive/Excel/zdp_acc2.csv', zdp_acc, fmt='%s')

In [ ]:
# Read saved csvs
epsilons = [0.1, 1.0, 10.0, 50.0, 100.0, 200.0]

dp_acc = pd.read_csv('/content/drive/My Drive/Excel/dp_acc2.csv', header=None)
rdp_acc = pd.read_csv('/content/drive/My Drive/Excel/rdp_acc2.csv', header=None)
zdp_acc = pd.read_csv('/content/drive/My Drive/Excel/zdp_acc2.csv', header=None)


In [ ]:
# Plot graph for different values of Epsilon
plt.figure(figsize=(6, 3), facecolor='0.9')
plt.plot(epsilons,dp_acc,  color="#4169e1", label="DP");
plt.plot(epsilons,rdp_acc,  color="#CCCCFF", label="RDP");
plt.plot(epsilons,zdp_acc, color = "#DE3163",  label="ZDP");
plt.legend()
plt.xlabel('epsilon')
plt.ylabel('accuracy')
plt.show();